# Chapter 7 — Multi-Agent: Sequential

Agents run **one after another**, passing state between them using `SequentialAgent`.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Define a Sequential Pipeline

Three agents in sequence: **Researcher → Writer → Editor**

In [ ]:
from google.adk.agents import Agent, SequentialAgent

researcher = Agent(
    name="researcher",
    model="gemini-2.0-flash",
    description="Gathers key facts and data about a topic.",
    instruction="Research the given topic and compile key facts, statistics, and talking points. Store your research in the output.",
    output_key="research_notes",
)

writer = Agent(
    name="writer",
    model="gemini-2.0-flash",
    description="Writes a draft article from research notes.",
    instruction="Using the research notes from the 'research_notes' session state, write a well-structured draft article. Make it engaging and informative.",
    output_key="draft_article",
)

editor = Agent(
    name="editor",
    model="gemini-2.0-flash",
    description="Edits and polishes the draft article.",
    instruction="Review and edit the draft from 'draft_article' in session state. Fix grammar, improve clarity, and ensure the tone is professional. Output the final polished article.",
)

pipeline = SequentialAgent(
    name="article_pipeline",
    sub_agents=[researcher, writer, editor],
)

print("Sequential pipeline: researcher → writer → editor")

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=pipeline, app_name="sequential_demo", session_service=session_service)

session = await session_service.create_session(app_name="sequential_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="Write an article about the impact of AI on healthcare.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text[:300]}...")
        print("---")

## Key Takeaways

- **`SequentialAgent`**: Runs sub-agents in order, each building on the previous agent's output
- **`output_key`**: Stores each agent's output in session state for the next agent
- Great for assembly-line workflows (research → draft → edit)